In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("storage-account", "adslproyecto01")
dbutils.widgets.text("esquema_source", "bronze")
dbutils.widgets.text("esquema_sink", "silver")

In [0]:
##Definicion de parametros
storage_account = dbutils.widgets.get("storage-account")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")
archivo_b3 = "reporte_b3"

In [0]:
def mayor_15min(tiempo_espera):
    if int(tiempo_espera) >= 900:
        return "Si"
    else:
        return "No"

In [0]:
tiempo_espera_15min = F.udf(mayor_15min, StringType())

In [0]:
df_reporte_b3 = spark.read \
    .option("header", True) \
    .csv(f"abfss://{esquema_source}@{storage_account}.dfs.core.windows.net/reporte_b3/")

In [0]:
df_reporte_b3_notnull = df_reporte_b3.dropna(how="all").filter((col("nomb_usuario").isNotNull()))


In [0]:
df_maestra_catalogo = spark.read \
    .option("header", True) \
    .csv(f"abfss://{esquema_source}@{storage_account}.dfs.core.windows.net/catalogo_maestras/")

In [0]:
df_maestra_m3 = df_maestra_catalogo.filter(col("codigo_catalogo") == "M_3")
df_maestra_m5 = df_maestra_catalogo.filter(col("codigo_catalogo") == "M_5")


In [0]:
df_reporte_b3_joined = df_reporte_b3_notnull.alias("x") \
                                    .join(df_maestra_m3.alias("y3"), col("x.tipo_solicitud") == col("y3.codigo"), "inner") \
                                    .join(df_maestra_m5.alias("y5"), col("x.tramite") == col("y5.codigo"), "inner") \
                                    .select("x.cod_oficina",
                                            "x.cod_arribo",
                                            "x.num_usuario",
                                            "x.nomb_usuario",
                                            "x.fec_inicio",
                                            "x.tipo_solicitud",
                                            col("y3.etiqueta").alias("desc_tipo_solicitud"),
                                            "x.tramite",
                                            col("y5.etiqueta").alias("desc_tramite"),
                                            "x.servicio",
                                            "x.fec_solicitado",
                                            "x.estado",
                                            "x.tiempo_espera",
                                            "x.fecha_registro")


In [0]:
df_reporte_b3_15min = df_reporte_b3_joined.withColumn("espera_mayor_15min", tiempo_espera_15min("tiempo_espera"))

In [0]:
df_reporte_b3_anio_mes = (df_reporte_b3_15min
                            .withColumn("fec_solicitado_ts",F.to_timestamp("fec_solicitado", "M/d/yyyy h:mm:ss a"))
                            .withColumn("año", F.year("fec_solicitado_ts"))
                            .withColumn("mes", F.month("fec_solicitado_ts"))
)

In [0]:
df_reporte_b3_final = df_reporte_b3_anio_mes.select("*") \
                                            .drop(col("fec_solicitado_ts"),
                                                  col("tiempo_espera"),
                                                  col("fec_solicitado"),
                                                  col("tipo_solicitud"),
                                                  col("tramite"))

In [0]:
df_reporte_b3_final.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(f"abfss://{esquema_sink}@{storage_account}.dfs.core.windows.net/reporte_b3_silver")